In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'aifrauddetector-donotdelete-pr-3a3ijgndt8rfpu'
object_key = 'data_asset/Synthetic_Financial_datasets_log_csv_shaped_1jcgpj180xzpfzzb70hgepbk4'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,0,181.00,181.00,0.0,0.0,0.00,1
1,1,1,181.00,181.00,0.0,21182.0,0.00,1
2,1,1,229133.94,15325.00,0.0,5083.0,51513.44,0
3,1,0,215310.30,705.00,0.0,22425.0,0.00,0
4,1,0,311685.89,10835.00,0.0,6267.0,2719172.89,0
5,1,1,110414.71,26845.41,0.0,288800.0,2415.16,0
6,1,1,56953.90,1942.02,0.0,70253.0,64106.18,0
7,1,1,23261.30,20411.53,0.0,25742.0,0.00,0
8,1,0,62610.80,79114.00,16503.2,517.0,8383.29,0
9,1,1,82940.31,3017.87,0.0,132372.0,49864.36,0


In [3]:
from sklearn.model_selection import train_test_split
train, holdout = train_test_split(df_1, test_size=0.1, random_state=42)
holdout.to_csv("holdout_dataset.csv", index=False)


In [4]:
# Save holdout_dataset.csv as an asset to the project

from ibm_watsonx_ai import APIClient

wml_credentials = {
    "apikey": API_KEY,
    "url": "https://us-south.ml.cloud.ibm.com"
}

client = APIClient(wml_credentials)

# Get the project ID (you can find it in project settings in the UI)
client.set.default_project("963792bf-30d2-4668-828e-fcfa6528c3b6")

# Store the local CSV as a data asset
client.data_assets.create(
    name="Holdout dataset",
    file_path="holdout_dataset.csv"
)


Creating data asset...
SUCCESS


{'metadata': {'project_id': '963792bf-30d2-4668-828e-fcfa6528c3b6',
  'sandbox_id': '963792bf-30d2-4668-828e-fcfa6528c3b6',
  'usage': {'last_updated_at': '2025-10-22T21:34:26Z',
   'last_updater_id': 'IBMid-693000IW98',
   'last_update_time': 1761168866832,
   'last_accessed_at': '2025-10-22T21:34:26Z',
   'last_access_time': 1761168866832,
   'last_accessor_id': 'IBMid-693000IW98',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'IBMid-693000IW98': {'user_iam_id': 'IBMid-693000IW98',
     'roles': ['OWNER']}}},
  'is_linked_with_sub_container': False,
  'name': 'Holdout dataset',
  'description': '',
  'asset_type': 'data_asset',
  'origin_country': 'us',
  'resource_key': 'Holdout dataset',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': 'c3d2165c-374c-4fc1-b9a6-97ebdedcceec',
  'created': 1761168866832,
  'created_at': '2025-10-22T21:34:26Z',
  'owner_id': 'IBMid-693000IW98',
  'size': 0,
  'version': 2.0,
  'asset_state': 'availabl